Takes files like number 2 in *file_process* and generates files like number 3. Combines functions which: reconstructed the time values to fill gaps of chosen length; calculated the angle between 2 consecutive segments created by joining points for a chosen plane; calculated the tortuosity of the 3d path (an older and a newer version are present in the file, v2 was used).

The loop in the final cells iterates over all files in a directory to apply these functions and then creates new files with all the outputs.

In [2]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import UnivariateSpline as US

In [3]:
def generate_time(og_time, step = 1/30, max_gap = 1.5):
    """
    Function that takes in an existing time series array, and creates a new array based on it.
    og_time is the existing time series data,
    step is the the desired gap between consecutive time steps,
    max_gap is the maximum gap between consecutive time steps that will be overwritten with newly generated steps. If max_gap is exceeded, the gap between points is left as it was.
    This function also adds two points extending into the "past" and "future" compared to the original data.
    """

    new_time = [og_time[0] - (step * 2), og_time[0] - step]

    for i in range(len(og_time) - 1):
        new_time.append(og_time[i])
        gap = og_time[i + 1] - og_time[i]

        #Looks for gaps largee than max_gap
        if gap <= max_gap:
            steps = int(gap / step) #Generates points within the gap with steps of step
            for j in range(1, steps):
                new_time.append(og_time[i] + j * step)

    new_time.append(og_time[-1])
    new_time.extend((og_time[-1] + step, og_time[-1] + (step * 2)))

    return np.array(new_time)

In [4]:
def calculate_plane_ang_change(x, y):
    """
    Given two spatial coordinates, calculates the change in angle on the plane created by these two coordinates.
    So angle in the:
    x-direction given y and z coordinates;
    y-direction given x and z coordinates;
    z-direction given x and y coordinates.
    """

    angles = []

    for i in range(2, len(x)):
        #Creates vectors in the plane
        v1 = np.array([x[i] - x[i - 1], y[i] - y[i - 1]])
        v2 = np.array([x[i - 1] - x[i - 2], y[i - 1] - y[i - 2]])

        #Calculates magnitudes of vectors
        mag1 = np.linalg.norm(v1)
        mag2 = np.linalg.norm(v2)

        #Calculates dot_product
        dot_product = np.dot(v1, v2)

        #Calculates the cosine of the angle
        cos = dot_product / (mag1 * mag2) if mag1 != 0 and mag2 != 0 else 1

        #Ensure value is in good range for arccos
        cos = np.clip(cos, -1, 1)

        #Calculates the angle and converts to degrees
        angle = np.degrees(np.arccos(cos))
        angles.append(angle)

    return np.array(angles)

In [11]:
def tortuosity(x, y, z, time, max_gap = 1.5):
    """
    Calculates the tortuosity of the path given three spatial coordinates x,y and z.
    time is an array with time steps of the spatial data. This is necessary because there are jumps in data when, for example, an experiment was restarted.
    max_gap is an optional variable that specifies how large of a gap to allow to continue the calculation. If max_gap is exceeded the calculation restarts from that point.
    """

    tort_values = []
    path_length = 0
    index = 0

    for i in range(1, len(x)):
        #Checks for a time gap larger than max_gap
        time_gap = time[i] - time[i - 1]

        if time_gap > max_gap:
            #Calculates displacement and tortuosity for the segment
            path_length = 0
            index = i
            tort_values.append(0)
            if index != i - 1:
                displacement = np.sqrt((x[i - 1] - x[index])**2 + (y[i - 1] - y[index])**2 + (z[i - 1] - z[index])**2)

                if displacement != 0:
                    tortuosity = path_length / displacement
                    tort_values.append(tortuosity)
            #Resets for next segment
            path_length = 0
            index = i
        
        else:
            #Calculates distance between consecutive points
            distance = np.sqrt((x[i] - x[i -  1])**2 + (y[i] - y[i - 1])**2 + (z[i] - z[i - 1])**2)
            path_length += distance

    #Calculates tortusoity for final segment
    if index < len(x) - 1:
        displacement = np.sqrt((x[-1] - x[index])**2 + (y[-1] - y[index])**2 + (z[-1] - z[index])**2)

        if displacement != 0:
            tortuosity = path_length / displacement
            tort_values.append(tortuosity)

    return np.array(tort_values)

In [23]:
def tortuosityv2(time, x, y, z, max_gap = 1.5):
    """
    Calculates the tortuosity of the path given three spatial coordinates x,y and z.
    time is an array with time steps of the spatial data. This is necessary because there are jumps in data when, for example, an experiment was restarted.
    max_gap is an optional variable that specifies how large of a gap to allow to continue the calculation. If max_gap is exceeded the calculation restarts from that point.
    Utilizes Pandas dataframes to improve handling of array operations.
    """

    #Creates a dataframe
    df = pd.DataFrame({
        "t": time,
        "x": x,
        "y": y,
        "z": z
    })

    #Calculates time difference between consecutive points
    df["time_diff"] = df["t"].diff()

    #Finds indexes where gaps between time steps exceed max_gap
    gaps = df[df["time_diff"] > max_gap].index.tolist()

    #Initializes an array to store tortuosity and index of first segment
    tort = []
    index = 0

    #Iterates through segments
    for i in gaps + [len(df)]: #Adds len(df) to handle the last segment
        segment = df.iloc[index : i]

        #initializes tortuosity array for segment
        segment_tort = [0] #First value chosen to be 0 to avoid NaNs

        #Calculates tortuosity
        for j in range(1, len(segment)):

            #Ensures that the segment has more than one point
            if len(segment) > 1:

                #Path length start at current point
                path_length = np.sum(np.sqrt(np.diff(segment["x"].iloc[:j + 1])**2 + np.diff(segment["y"].iloc[:j + 1])**2 + np.diff(segment["z"].iloc[:j + 1])**2))

                #straight-line distance from start to current position
                distance = np.sqrt((segment["x"].iloc[j] - segment["x"].iloc[0])**2 + (segment["y"].iloc[j] - segment["y"].iloc[0])**2 + (segment["y"].iloc[j] - segment["y"].iloc[0])**2)

                #Avoids division by 0
                if distance == 0:
                    tort_value = 0
                else:
                    tort_value = path_length / distance
            else:
                tort_value = 0

            segment_tort.append(tort_value)

        #Appends segments to full list
        tort.extend(segment_tort)

        #Updates start index for following segment
        index = i

    return np.array(tort)


In [6]:
read_directory = "/Users/maks/Documents/MSc_project/data/interpolation_prep"
write_directory = "/Users/maks/Documents/MSc_project/data/variables_data"

In [31]:
#iterates over files with data prepared for interpolation and calculation of variables we are interested in 
for filename in os.listdir(read_directory):
    if filename.startswith("."): #ignores hidden files
        continue
    elif filename.endswith(".xlsx"): #allows only Excel files
        print(filename) #shows which file is worked on

        input = os.path.join(read_directory, filename)

        df = pd.read_excel(input, usecols = "A:D", engine = "openpyxl")

        #Extracts necessary data
        time = df["time"].values
        x = df["x"].values
        y = df["y"].values
        z = df["z"].values

        #Performs interpolation
        spline_x = US(time, x, k = 3, s = 0)
        spline_y = US(time, y, k = 3, s = 0)
        spline_z = US(time, z, k = 3, s = 0)

        #Generates new time array with generate_time
        new_time = generate_time(time)

        #Calculates interpolated positions
        new_x = spline_x(new_time)
        new_y = spline_y(new_time)
        new_z = spline_z(new_time)

        #Prepeares derivatives for velocity calculations
        derivative_x = spline_x.derivative()
        derivative_y = spline_y.derivative()
        derivative_z = spline_z.derivative()

        #Calculates velocities
        v_x = derivative_x(new_time)
        v_y = derivative_y(new_time)
        v_z = derivative_z(new_time)

        #Calculates tortuosity using tortuosity
        tort = tortuosityv2(new_time, new_x, new_y, new_y)

        #Calculates angles using calculate_plane_ang_change and later the derivative to get angular velocities

        #x-direction
        ang_yz = calculate_plane_ang_change(new_y, new_z)
        ang_v_yz = np.gradient(ang_yz)

        #y-direction
        ang_xz = calculate_plane_ang_change(new_x, new_z)
        ang_v_xz = np.gradient(ang_xz)

        #z-direction
        ang_xy = calculate_plane_ang_change(new_x, new_y)
        ang_v_xy = np.gradient(ang_xy)

        #Prepares some arrays for saving by making them the same length
        final_time = new_time[1 : -1]
        final_x = new_x[1 : -1]
        final_y = new_y[1 : -1]
        final_z = new_z[1 : -1]
        final_vx = v_x[1 : -1]
        final_vy = v_y[1 : -1]
        final_vz = v_z[1 : -1]
        final_tort = tort[1 : -1]

        #print(len(final_time))
        #print(len(final_x))
        #print(len(final_y))
        #print(len(final_z))
        #print(len(final_vx))
        #print(len(final_vy))
        #print(len(final_vz))
        #print(len(final_tort))
        #print(len(ang_v_xy))
        #print(len(ang_v_xz))
        #print(len(ang_v_yz))
        #print(len(ang_yz))
        #print(len(ang_xz))
        #print(len(ang_xy))

        #Creates a dataframe and saves to Excel
        output_df = pd.DataFrame({
            "time": final_time,
            "x": final_x,
            "y": final_y,
            "z": final_z,
            "velocity_x": final_vx,
            "velocity_y": final_vy,
            "velocity_z": final_vz,
            "tortuosity3d": final_tort,
            "angular_velocity_yz": ang_v_yz,
            "angular_velocity_xz": ang_v_xz,
            "angular_velocity_xy": ang_v_xy,
            "angle_yz": ang_yz,
            "angle_xz": ang_xz,
            "angle_xy": ang_xy
        })


        output_path = os.path.join(write_directory, os.path.basename(input))
        output_df.to_excel(output_path, index = False)

        print(f"Calculated variables saved to {output_path}")





yellow85.xlsx
Calculated variables saved to /Users/maks/Documents/MSc_project/data/variables_data/yellow85.xlsx
untagged2.xlsx
Calculated variables saved to /Users/maks/Documents/MSc_project/data/variables_data/untagged2.xlsx
whiter73.xlsx
Calculated variables saved to /Users/maks/Documents/MSc_project/data/variables_data/whiter73.xlsx
yellow66.xlsx
Calculated variables saved to /Users/maks/Documents/MSc_project/data/variables_data/yellow66.xlsx
blue41.xlsx
Calculated variables saved to /Users/maks/Documents/MSc_project/data/variables_data/blue41.xlsx
yellow71.xlsx
Calculated variables saved to /Users/maks/Documents/MSc_project/data/variables_data/yellow71.xlsx
untagged3.xlsx
Calculated variables saved to /Users/maks/Documents/MSc_project/data/variables_data/untagged3.xlsx
blue83.xlsx
Calculated variables saved to /Users/maks/Documents/MSc_project/data/variables_data/blue83.xlsx
green62.xlsx
Calculated variables saved to /Users/maks/Documents/MSc_project/data/variables_data/green62.xls